In [1]:
import pandas as pd
import numpy as np

# fancyimpute

In [2]:
!pip install fancyimpute

In [3]:
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

In [4]:
X = np.random.rand(200, 5)
# make some gaps in data
X_orig = X.copy()
assert not np.isnan(X_orig.sum())  # niciun nan
lines = np.random.choice(X.shape[0], 10)
cols = np.random.choice(X.shape[1], 10)
X[lines, cols] = np.nan
assert np.isnan(X.sum()) # cel putin un nan in X

In [5]:
X_filled_knn = KNN(k=3).fit_transform(X)
assert not np.isnan(X_filled_knn.sum())

Imputing row 1/200 with 0 missing, elapsed time: 0.006
Imputing row 101/200 with 0 missing, elapsed time: 0.006


# missingpy

In [6]:
!pip install missingpy

In [7]:
# trick from https://stackoverflow.com/questions/60145652/no-module-named-sklearn-neighbors-base

import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [8]:
from missingpy import MissForest

In [9]:
assert np.isnan(X.sum())  # niciun nan

In [10]:
imputer = MissForest()

In [11]:
X_imputed = imputer.fit_transform(X)

Iteration: 0
Iteration: 1
Iteration: 2


In [12]:
assert not np.isnan(X_imputed.sum())  # nu exista nan in X_imputed